# Description

In this demo we will look at the cost of a sparse matrix-vector multipy ($A * v$) for different sparse matrix formats.

In [ ]:
from pyamg import gallery
import numpy as np
from time import time
import matplotlib.pyplot as plt
from common import set_figure
set_figure(width=600)

# Set up problem sizes

Set up the tests to do `ntrials` of $w \leftarrow A * v$ at different sizes in `nlist` with `nnzperrow` non-zeros per row on average

In [ ]:
types = ['csr', 'csc', 'coo', 'lil']
nlist = np.logspace(2,4,20, dtype=int)
ntrials = 10
nnzperrow = 50
print(nlist)

## set up containers

In [ ]:
times = {a: np.zeros((len(nlist), ntrials)) for a in types}
A = {a: [] for a in types}
nnz = []

## create matrix and time it

In [ ]:
for j, n in enumerate(nlist):
    print("n = %d" % n)
    # Generate a random matrix
    tmp = time()
    A['csr'] = gallery.sprand(n, n, float(nnzperrow) / n, format='csr')
    tmp = time() - tmp
    print("    setup: %g" % tmp)
    
    # initialize the input/output vectors
    v = np.random.rand(n,)
    w = 0 * v.copy()
    for tp in types:
        if tp == 'csr':
            nnz.append(A[tp].nnz)
        if tp != 'csr':
            A[tp] = getattr(A['csr'], 'to' + tp)()
        
        # time it for ntrials
        for i in range(ntrials):
            tmp = time()
            w = A[tp] * v
            tmp = time() - tmp
            times[tp][j, i] = tmp
        print("    mat-vec %s: %g" % (tp, tmp))

# Plot problem size (nnz) and times in log-log

In [ ]:
nnz = np.array(nnz)
for tp in types:
    plt.loglog(nlist, times[tp].min(axis=1), label=tp, lw=3)
plt.loglog(nlist, nnz / nnz[0] * times['csr'][0, 0], label='nnz')
plt.xlabel('size, nnz')
plt.ylabel('time (s)')
plt.grid(True)
plt.legend()
plt.show()